In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.append("/home/ubuntu/Notebooks/annsa/")

from scipy.interpolate import griddata
import annsa as an
import numpy as np

/home/ubuntu/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Define neural network

In [2]:
from cvt_oct_load_templates import *

### Load normalized templates, set isotope list and shielding settings

In [3]:
spectral_templates=load_templates(normalization='normalarea')



isotope_list=an.isotopes[:-3]

125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values
125I Contains no values


cvt_oct_load_templates.py:34: RuntimeWarning: invalid value encountered in divide
  return temp_spectrum/np.sum(temp_spectrum)


## Define simulation function

In [4]:
def simulate_shielded_template_dataset(isotope_list,
                                       spectral_template_settings,
                                       integration_times,
                                       signal_to_backgrounds,
                                       calibrations):

    all_source_spectra=[]
    all_keys=[]
    LLD=10
    background_cps=85.
    total_spectra=0
    random_settings=False

    for isotope in isotope_list:
        for spectral_template_setting in spectral_template_settings:
            for integration_time in integration_times:
                for signal_to_background in signal_to_backgrounds:
                    for calibration in calibrations:
                        isotopes_per_setting=1
                        # Simulate extra unshielded spectra to avoid training set imbalance 
                        if spectral_template_setting=='noshield':
                            isotopes_per_setting*=len(spectral_template_settings)-1
                        for _ in range(isotopes_per_setting):


                            # I125 emits a maximum gamma-ray energy of 30keV. 
                            # This will either be completely attenuated by any shielding, or produce insignificatn Compton Continuum.
                            # For this reason shielded I125 is removed as a class
                            if isotope=='I125' and spectral_template_setting!='noshield':
                                continue

                            # Simulate source
                            if random_settings==True:
                                calibration=np.random.uniform(calibrations[0],calibrations[-1])
                                signal_to_background=10**np.random.uniform(np.log10(signal_to_backgrounds[0]),np.log10(signal_to_backgrounds[-1]))
                                integration_time=10**np.random.uniform(np.log10(integration_times[0]),np.log10(integration_times[-1]))


                            source_template=spectral_templates[spectral_template_setting][isotope]
                            source_template=griddata(range(1024), source_template, calibration*np.arange(1024),method='cubic',fill_value=0.0)
                            source_template[0:LLD]=0
                            source_template[source_template < 0] = 0
                            source_template/=np.sum(source_template)
                            source_template*=integration_time*background_cps*signal_to_background


                            background_template=spectral_templates['background']['chicago']
                            background_template=griddata(range(1024), background_template, calibration*np.arange(1024),method='cubic',fill_value=0.0)
                            background_template[0:LLD]=0
                            background_template[background_template < 0] = 0
                            background_template/=np.sum(background_template)
                            background_template*=integration_time*background_cps

                            """
                            flag=0
                            while np.sum(np.isnan(sampled_spectrum))!=0:
                                sampled_spectrum=an.sample_spectrum(source_template,
                                               generate_random_counts_on_detector(integration_time))
                                flag+=1
                                if flag==1:
                                    print 'spectral template'+spectral_templates[spectral_template_setting]+'contains NaN'
                                    break
                            """
                            all_source_spectra.append(source_template+background_template)

                            isotope_key=isotope

                            if spectral_template_setting!='noshield':
                                isotope_key+='_shielded'
                            all_keys.append(isotope_key)

                            print ('\1b[2k\r'),    
                            print('Isotope %s, template %s, %s total spectra simulated' %(isotope,
                                                                                          spectral_template_setting,
                                                                                          total_spectra)),



    return np.array(all_source_spectra),np.array(all_keys)

## Final CVT-OCT dataset divisions

In [9]:
spectral_template_settings=['noshield',
                           'aluminum40pct',
                           'aluminum60pct', 
                           'aluminum80pct',
                           'iron40pct',
                           'iron60pct',
                           'iron80pct',
                           'lead40pct',
                           'lead60pct',
                           'lead80pct']

integration_time_division=6
signal_to_background_division=6
calibration_division=7

integration_times=np.logspace(np.log10(10),np.log10(3600),integration_time_division)
signal_to_backgrounds=np.logspace(np.log10(0.25),np.log10(4),signal_to_background_division)
calibrations=np.linspace(1.19*0.9,1.19*1.1,calibration_division)

print integration_times
print signal_to_backgrounds
print calibrations

[  10.           32.45342223  105.32246146  341.80743122 1109.28208873
 3600.        ]
[0.25       0.43527528 0.75785828 1.31950791 2.29739671 4.        ]
[1.071      1.11066667 1.15033333 1.19       1.22966667 1.26933333
 1.309     ]


In [10]:
training_data,training_keys=simulate_shielded_template_dataset(isotope_list,
                                                               spectral_template_settings,
                                                               integration_times,
                                                               signal_to_backgrounds,
                                                               calibrations)

print training_data.shape

Isotope U235, template lead80pct, 0 total spectra simulated (129276, 1024)


In [11]:
np.save('FINAL_template_training_data.npy',training_data)
np.save('FINAL_template_training_keys.npy',training_keys)

## CVT-OCT Hyperparameter training dataset divisions

In [89]:
spectral_template_settings=['noshield',
                           'aluminum40pct',
                           'aluminum80pct',
                           'iron40pct',
                           'iron80pct',
                           'lead40pct',
                           'lead80pct']

integration_time_division=4
signal_to_background_division=4
calibration_division=4

integration_times=np.logspace(np.log10(60),np.log10(1800),integration_time_division)
signal_to_backgrounds=np.logspace(np.log10(0.5),np.log10(3),signal_to_background_division)
calibrations=np.linspace(1.19*0.9,1.19*1.1,calibration_division)

print integration_times
print signal_to_backgrounds
print calibrations

 [  60.          186.43395036  579.29363076 1800.        ]
[0.5        0.9085603  1.65096362 3.        ]
[1.071      1.15033333 1.22966667 1.309     ]


In [90]:
training_data,training_keys=simulate_shielded_template_dataset(isotope_list,
                                                               spectral_template_settings,
                                                               integration_times,
                                                               signal_to_backgrounds,
                                                               calibrations)

print training_data.shape

Isotope U235, template lead80pct, 0 total spectra simulated (21888, 1024)


In [91]:
np.save('FINAL_template_hyperparameter_training_data.npy',training_data)
np.save('FINAL_template_hyperparameter_training_keys.npy',training_keys)

## CVT-OCT Hyperparameter testing dataset divisions

In [87]:
spectral_template_settings=['noshield',
                           'aluminum40pct',
                           'aluminum80pct',
                           'iron40pct',
                           'iron80pct',
                           'lead40pct',
                           'lead80pct']

integration_times=np.logspace(np.log10(60),np.log10(1800),integration_time_division)
signal_to_backgrounds=np.logspace(np.log10(0.5),np.log10(3),signal_to_background_division)
calibrations=np.linspace(1.19*0.9,1.19*1.1,calibration_division)

integration_times=integration_times[:-1]+np.diff(integration_times)/2.0
signal_to_backgrounds=signal_to_backgrounds[:-1]+np.diff(signal_to_backgrounds)/2.0
calibrations=calibrations[:-1]+np.diff(calibrations)/2.0

print integration_times
print signal_to_backgrounds
print calibrations

[ 123.21697518  382.86379056 1189.64681538]
[0.70428015 1.27976196 2.32548181]
[1.11066667 1.19       1.26933333]


In [ ]:
training_data,training_keys=simulate_shielded_template_dataset(isotope_list,
                                                               spectral_template_settings,
                                                               integration_times,
                                                               signal_to_backgrounds,
                                                               calibrations)

print training_data.shape